In [88]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from mlb.Data import Data

In [240]:
df = pd.read_csv('data/games.csv')

In [247]:
import pandas as pd
import numpy as np
import re
from typing import List


def normalize_columns(cols: pd.Index) -> List[str]:
    """
    Normalize column names by converting to lowercase and replacing special characters with underscores.

    Args:
        cols: Pandas Index of column names to normalize

    Returns:
        List of normalized column names
    """
    replace_dict = {
        ' - ': '_',
        '-': '_',
        ' ': '_',
        '/': '_'
    }

    new_cols = []
    for col in cols:
        temp = col.lower()
        for old_char, new_char in replace_dict.items():
            temp = temp.replace(old_char, new_char)
        new_cols.append(temp)

    return new_cols


def _process_record_column(df: pd.DataFrame, col: str) -> pd.DataFrame:
    """
    Process individual record columns (wins-losses format).

    Args:
        df: DataFrame containing the record column
        col: Column name to process

    Returns:
        DataFrame with wins and losses columns added, original column removed
    """
    df_result = df.copy()

    if 'pitcher' in col:
        # Handle pitcher records: remove parentheses first
        df_result[col] = df_result[col].str.replace(r'\((\d+-?\d*)\)', r'\1', regex=True)

        if 'save' in col:
            # Special handling for save records - extract saves count and create saves column
            df_result[col.replace('record', 'saves')] = df_result[col].str.replace(r'\((\d+)\)', r'\1', regex=True)
            df_result = df_result.drop(col, axis=1)
            return df_result

    # Extract wins and losses for all other record columns
    wins_col = col.replace('record', 'wins')
    losses_col = col.replace('record', 'losses')

    df_result[wins_col] = df_result[col].str.replace(r'(\d+)-{1}(\d+)\s*\w*', r'\1', regex=True)
    df_result[losses_col] = df_result[col].str.replace(r'(\d+)-{1}(\d+)\s*\w*', r'\2', regex=True)

    df_result = df_result.drop(col, axis=1)
    return df_result


def _process_stats_column(df: pd.DataFrame, col: str) -> pd.DataFrame:
    """
    Process individual stats columns (IP, ER, K, BB format).

    Args:
        df: DataFrame containing the stats column
        col: Column name to process

    Returns:
        DataFrame with parsed stats columns added, original column removed
    """
    df_result = df.copy()

    if 'save' in col:
        # Fill missing save stats with default values
        df_result[col] = df_result[col].fillna('0.0 IP, 0 ER, 0 K, 0 BB')

    # Parse stats using regex pattern
    stat_regex = r'(\d)\.(\d) IP, (\d+) ER, (\d+) K, (\d+) BB'

    # Extract full innings and partial outs
    df_result[col.replace('stats', 'full')] = df_result[col].str.replace(stat_regex, r'\1', regex=True).apply(lambda x: int(x))
    df_result[col.replace('stats', 'partial')] = df_result[col].str.replace(stat_regex, r'\2', regex=True).apply(lambda x: int(x))

    # Calculate total outs (3 outs per inning + partial outs)
    df_result[col.replace('stats', 'outs')] = (df_result[col.replace('stats', 'full')] * 3 +
                                              df_result[col.replace('stats', 'partial')])

    # Remove temporary columns
    df_result = df_result.drop([col.replace('stats', 'full'), col.replace('stats', 'partial')], axis=1)

    # Extract other statistics
    df_result[col.replace('stats', 'er')] = df_result[col].str.replace(stat_regex, r'\3', regex=True).apply(lambda x: int(x))
    df_result[col.replace('stats', 'k')] = df_result[col].str.replace(stat_regex, r'\4', regex=True).apply(lambda x: int(x))
    df_result[col.replace('stats', 'bb')] = df_result[col].str.replace(stat_regex, r'\5', regex=True).apply(lambda x: int(x))

    # Remove original stats column
    df_result = df_result.drop(col, axis=1)
    return df_result


def _process_attendance_capacity(df: pd.DataFrame, col: str) -> pd.DataFrame:
    """
    Process attendance and capacity columns by removing commas and converting to numeric.

    Args:
        df: DataFrame containing the column
        col: Column name to process

    Returns:
        DataFrame with processed numeric column
    """
    df_result = df.copy()

    # Remove commas and convert to numeric
    df_result[col] = df_result[col].str.replace(',', '')
    df_result[col] = pd.to_numeric(df_result[col])

    # Fill missing values with mean (as integer)
    df_result[col] = df_result[col].fillna(int(df_result[col].mean()))

    return df_result


def _process_duration(df: pd.DataFrame, col: str) -> pd.DataFrame:
    """
    Process duration column by converting from H:MM format to total minutes.

    Args:
        df: DataFrame containing the duration column
        col: Column name to process

    Returns:
        DataFrame with duration in minutes
    """
    df_result = df.copy()

    # Extract hours and minutes
    df_result['hours'] = df_result[col].str.split(':').apply(lambda x: int(x[0]))
    df_result['minutes'] = df_result[col].str.split(':').apply(lambda x: int(x[1][:2]))

    # Convert to total minutes
    df_result[col] = df_result['minutes'] + 60 * df_result['hours']

    # Remove temporary columns
    df_result = df_result.drop(['minutes', 'hours'], axis=1)

    return df_result


def clean_games(df: pd.DataFrame) -> pd.DataFrame:
    """
    Comprehensive cleaning of MLB games dataset with exact functionality as original.

    Args:
        df: Raw MLB games DataFrame

    Returns:
        Cleaned DataFrame ready for analysis
    """
    # Work with a copy to avoid modifying original
    df_clean = df.copy()

    # Set game ID as index and sort by date
    df_clean = df_clean.set_index('game')
    df_clean = df_clean.sort_values('date')

    # Reorder columns: date, home, away first, then everything else
    df_clean = pd.concat([
        df_clean[['date', 'home', 'away']],
        df_clean.drop(['date', 'home', 'away'], axis=1)
    ], axis=1)

    # Remove duplicate rows
    df_clean = df_clean.drop(df_clean[df_clean.duplicated()].index)

    # Remove rows with missing values in columns that have fewer than 100 missing values
    for col in df_clean.columns:
        if df_clean[col].isna().sum() < 100:
            df_clean = df_clean.drop(df_clean[df_clean[col].isna()].index)

        # Special case: fill missing save pitcher records with default
        if col == 'save_pitcher_record':
            df_clean[col] = df_clean[col].fillna('(0)')

    # Process each column according to its type
    columns_to_process = list(df_clean.columns)  # Create a copy of column list

    for col in columns_to_process:

        # Handle record columns (wins-losses format)
        if 'record' in col:
            df_clean = _process_record_column(df_clean, col)
            continue

        # Handle postseason information
        if 'postseason' in col:
            df_clean['regular_season'] = df_clean[col].isna()
            df_clean[col] = df_clean[col].fillna('Regular Season')
            continue

        # Handle stadium column
        if col == 'stadium':
            df_clean[col] = df_clean[col].str.replace(r'^\\s*(.+(\\s\\w+)*){1}(.*\\s*)*', r'\\1', regex=True)
            continue

        # Handle location column
        if col == 'location':
            df_clean[col] = df_clean[col].str.replace(r'^(.+(\\s\\w+)*)\\s*\\d*', r'\\1', regex=True)
            continue

        # Handle attendance and capacity columns
        if col in ['attendance', 'capacity']:
            df_clean = _process_attendance_capacity(df_clean, col)
            continue

        # Handle duration column
        if col == 'duration':
            df_clean = _process_duration(df_clean, col)
            continue

        # Handle stats columns (pitcher statistics)
        if 'stats' in col:
            df_clean = _process_stats_column(df_clean, col)
            continue

        # Handle extra innings column
        if 'extra' in col:
            df_clean[col] = df_clean[col].fillna(False)
            continue

    return df_clean


In [248]:
df.columns = normalize_columns(df.columns)
df_clean = clean_games(df)

C:\Users\theru\AppData\Local\Temp\ipykernel_42172\3820605844.py:236: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_clean[col] = df_clean[col].fillna(False)


In [249]:
df_clean

,date,home,away,away_score,home_score,postseason_info,walks_issued_away,walks_issued_home,stolen_bases_away,stolen_bases_home,...,loss_pitcher_er,loss_pitcher_k,loss_pitcher_bb,loss_pitcher_wins,loss_pitcher_losses,save_pitcher_outs,save_pitcher_er,save_pitcher_k,save_pitcher_bb,save_pitcher_saves
game,,,,,,,,,,,,,,,,,,,,,
360403123,2016-04-03T17:00Z,PIT,STL,1.0,4.0,Regular Season,5.0,5.0,0.0,0.0,...,3,3,3,0,1,0,0,0,0,0
360403130,2016-04-03T20:00Z,TB,TOR,5.0,3.0,Regular Season,1.0,3.0,0.0,0.0,...,2,12,3,0,1,3,0,2,0,1
360403107,2016-04-04T00:30Z,KC,NYM,3.0,4.0,Regular Season,2.0,6.0,0.0,1.0,...,3,2,2,0,1,3,0,2,1,1
360404108,2016-04-04T18:10Z,MIL,SF,12.0,3.0,Regular Season,5.0,5.0,1.0,0.0,...,4,2,2,0,1,0,0,0,0,0
360404101,2016-04-04T19:05Z,BAL,MIN,2.0,3.0,Regular Season,5.0,0.0,0.0,1.0,...,1,1,1,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401361977,2021-10-28T00:09Z,HOU,ATL,2.0,7.0,"World Series - Game 2, ATL Wins series 4-2",2.0,2.0,0.0,2.0,...,6,6,1,0,1,0,0,0,0,0
401361978,2021-10-30T00:09Z,ATL,HOU,0.0,2.0,"World Series - Game 3, ATL Wins series 4-2",5.0,3.0,1.0,0.0,...,1,6,4,0,1,3,0,0,0,1
401361979,2021-10-31T00:09Z,ATL,HOU,2.0,3.0,"World Series - Game 4, ATL Wins series 4-2",3.0,5.0,1.0,0.0,...,2,1,0,0,1,3,0,1,0,2
